[![launch on Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/slayoo/seattle-2026-workshop/blob/main/part_4_CCN_partial.ipynb)

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install open-atmos-jupyter-utils mendeleev pint PySDM

# hacking aerosol-cloud μ-physics modeling concepts in Python
### (mini-workshop @Uni. Washington Seattle, Jan 2026)

### Part 3: CCN activation model from scratch in Python (unit handling with Pint)

notebook authors (alphabetically): 
- Sylwester Arabas
- Daria Klimaszewska
- Emma Ware

This notebook implements a basic adiabatic air parcel model with moving-sectional (=super-droplet) aerosol-cloud spectrum description.
For reference on the employed theory and equations, see e.g. [Arabas & Shima 2017](https://doi.org/10.5194/npg-24-535-2017) (where the model is applied with a monodisperse droplet population. Note that the code below extends the model to a polydisperse case (to enable resolving the ripening proces), while for simplicity the size-spectrum is assumed to be sample with constant-multiplicity sampling.

### • representing a lognormal distribution of dry radii using deterministic constant-multiplicity sampling

In [ ]:
import numpy as np, scipy
from matplotlib import pyplot
from open_atmos_jupyter_utils import show_plot

σ_geom = 1.1
r_dry_median_metre = .05e-6
n_sd = 25

dry_radii_over_metre = scipy.stats.lognorm.ppf(
    np.linspace(.01, .99, n_sd),
    np.log(σ_geom),
    0,
    r_dry_median_metre
)

pyplot.hist(dry_radii_over_metre)
pyplot.gca().set(ylabel="super droplet count", xlabel="dry radius [m]")
show_plot()

### • constants (indices, fundamental/atmospheric physical constants, model parameters and initial state)

In [ ]:
from collections import namedtuple
make_namedtuple = lambda name, items: namedtuple(name, items.keys())(*items.values())

ix = make_namedtuple("I", {
    "ln_rw": # TODO
    "pd": # TODO
    "T": # TODO
    "size": # TODO
})
print(ix)

In [ ]:
from PySDM import physics
import mendeleev

def constants():
    SI = physics.si

    y0 = [0] * ix.size
    y0[ix.pd] = 1000 * SI.hPa
    y0[ix.T] = 300 * SI.K
    y0[ix.ln_rw] = # TODO
    
    M_a = (
        0.78 * 2 * mendeleev.N.atomic_weight * SI.g / SI.mole + 
        0.21 * 2 * mendeleev.O.atomic_weight * SI.g / SI.mole + 
        0.01 * 1 * mendeleev.Ar.atomic_weight * SI.g / SI.mole
    )
    M_v = 0 # TODO
    R_str = scipy.constants.R * SI.J / SI.K / SI.mole
    
    return make_namedtuple("C", {
        "R_d": R_str / M_a,
        "R_v": R_str / M_v,
        "cp_d": 1000 * SI.J / SI.kg / SI.K,
        "l_v": 45000 * SI.joule / SI.mol / M_v,
        "g": scipy.constants.g * SI.m / SI.s**2,
        "A": 1e-3 * SI.um,
        "rho_w": 1 * SI.kg / SI.litre,
        "D_eff": 2.3e-5 * SI.metre**2 / SI.s,
        "T0": scipy.constants.zero_Celsius * SI.K,

        "B80_G0": 6.112 * SI.hPa,
        "B80_G1": 17.67 * SI.dimensionless,
        "B80_G2": 243.5 * SI.K,
        
        "κ": 1.28,
        "w": 1 * SI.m / SI.s,
        "r_d": # TODO
        "xi": # TODO 
        "m_d": # TODO
        "qt": # TODO
        "t_max": # TODO
        "y0": # TODO
    }), SI

### • formulae expressing the math of the parcel-model (incl. diffusion of vapour around a droplet, hydrostatic and adiabatic conditions, ideal gas equation of state, relative humidity and size spectrum trivia)

In [ ]:
formulae = make_namedtuple("F", {
    # eq. (1) in the NPG paper
    "dr_dt": lambda c, rw, rho_v, rho_o: 0, # TODO
    # eq. (4) in the NPG paper
    "RH_eq": lambda c, rw, rd: 0, # TODO
    # eq. (13) in the NPG paper
    "dpd_dt": lambda c, rhod: 0, # TODO
    "dT_dt": lambda c, dpd_dt, dqv_dt, rhod: (dpd_dt / rhod - dqv_dt * c.l_v) / c.cp_d,
    
    # Bolton 1980 saturation vapour pressure formula (https://doi.org/10.1175/1520-0493(1980)108%3C1046:TCOEPT%3E2.0.CO;2)
    "pvs": lambda c, T: c.B80_G0 * np.exp((c.B80_G1 * (T - c.T0)) / ((T - c.T0) + c.B80_G2)),

    # trivia
    "rhod": lambda c, pd, T: pd / c.R_d / T,
    "rhov": lambda c, pv, T: pv / c.R_v / T,
    "RH": lambda c, rw3, rho_vs, rhod: rhod * (c.qt - c.xi * np.sum(rw3, axis=0) / c.m_d * 4/3 * np.pi * c.rho_w) / rho_vs,
    "dqv_dt": lambda c, rw, drw_dt: 0, # TODO
})

### • plotting the [Köhler curves](https://en.wikipedia.org/wiki/K%C3%B6hler_theory) for selected of the sampled super droplets

In [ ]:
from PySDM.physics.dimensional_analysis import DimensionalAnalysis

with DimensionalAnalysis():
    c, SI = constants()
    SI.setup_matplotlib()
    for rd in c.r_d[::n_sd//5]:
        pyplot.semilogx(
            rw := np.logspace(-.5, 2, 100) * SI.um,
            formulae.RH_eq(c, rw=rw, rd=rd),
            label=f"Köhler curve for $κ$={c.κ} and $r_\\text{{d}}$={rd.to(SI.um):.2g}"
        )
    pyplot.axhline(1, label="RH=100%", linestyle=':')
    pyplot.legend()
show_plot()

### • defining the ODE system constituting the adiabatic parcel model and JIT-compiling it using Numba

In [ ]:
import numba
jit_compile = numba.jit(error_model='numpy', fastmath=True)

In [ ]:
jf = make_namedtuple("J", {k: jit_compile(v) for k, v in formulae._asdict().items()})

@jit_compile
def ode_rhs(z, y, c):
    rhod = jf.rhod(c, pd=y[ix.pd], T=y[ix.T])
    rho_vs = # TODO
    rw = # TODO
    RH = # TODO
    RH_eq = # TODO
    
    dy_dt = np.empty_like(y)
    dy_dt[ix.pd] = 0 # TODO
    dy_dt[ix.ln_rw] = 0 # TODO
    dy_dt[ix.T] = 0 # TODO
    return dy_dt

### • integrating the ODE system using [SciPy's wrapper of the of Fortran LSODA solver from ODEPACK](https://docs.scipy.org/doc/scipy-1.17.0/reference/generated/scipy.integrate.solve_ivp.html) 

In [ ]:
c, SI = constants()
solution = scipy.integrate.solve_ivp(
    # TODO
)
assert solution.success, solution.message

### • sample visualisation of the solution (leveraging Pint's support for auto-labelling of matplotlib plots)

In [ ]:
with DimensionalAnalysis():
    c, SI = constants()
    SI.setup_matplotlib()

    t = solution.t * SI.s
    pd = solution.y[ix.pd] * SI.Pa
    T = solution.y[ix.T] * SI.K
    rw = np.exp(solution.y[ix.ln_rw]) * SI.m
    RH = formulae.RH(c,
                     rw3=rw**3,
                     rho_vs=formulae.rhov(c, pv=formulae.pvs(c, T), T=T),
                     rhod=formulae.rhod(c, pd=pd, T=T)
                    )
    
    pyplot.plot(RH - 1, t * c.w, label="supersaturation", color='red', linewidth=1)
    pyplot.axvline(0, label="RH=100%", linestyle=':', color='blue', linewidth=.5)
    pyplot.legend(loc='lower right')
    
    twin = pyplot.gca().twiny()
    for i in range(ix.ln_rw.start, ix.ln_rw.stop):
        twin.plot(rw[i], t * c.w, color='k', linewidth=.5)
    twin.xaxis.set_units(SI.um)
show_plot()

### • further reading on Ostwald ripening in clouds
- [Arenberg 1939 (J. Atmos. Sci.)](http://doi.org/10.1175/1520-0477-20.10.444) cited by: [Google Scholar](https://scholar.google.com/scholar?cites=8244002676805439446)  
  "_Turbulence As The Major Factor in the Growth of Cloud Drops_"   
  > By   taking   the   average    values    of    vapor    pressure,    mass   of   drops,   and   supersaturation   over  a  complete  cycle,  it  can  be  shown  that  the  larger   drops  will   require   a   somewhat  unsaturated  average  vapor  pressure  to  maintain  a  constant  size, while  the   smaller   drops  will   need  a  more    nearly    saturated    atmosphere.    In  a  mixture  of  the  two  drops,  both  will  pool  their  excess  vapor  in  their  common   environment   during   heating   and   later   extract   it   during   cooling.   Both  will  experience  the  same  degree  of   supersaturation  which  will  be  the  resultant   determined   by  the   relative   numbers  and  requirements  of  the  different  sizes.    Since  this  value  will  be  greater   than   that   demanded   by   the   large   drops   and   less   than   that   for   the  small  ones,  the  former  will  grow  at  the  expense  of  the  latter.
- [Hagen 1979](https://doi.org/10.1175/1520-0450%281979%29018%3C1035:ANCMFT%3E2.0.CO;2)   
  "_A Numerical Cloud Model for the Support of Laboratory Experimentation_"
  > Three types of experiments are treated: cloud formation, droplet growth and Ostwald ripening
- [Tsang & Brock 1984](https://doi.org/10.1080/02786828408959016)   
  "_On Ostwald Ripening_"
- [Tsang & Brock 1986](https://doi.org/10.1080/02786828608959103)   
  "_Dynamics of Ostwald Ripening with Coalescencefor Aerosols with Continuum Diffusive Growth Laws_"
- [Barrett & Clement 1988](https://doi.org/10.1016/0021-8502(88)90225-X)   
  "_Growth rates for liquid drops_"
  > We show that radiation can strongly influence droplet growth and, not only in the atmosphere
as was previously pointed out by Barkstrom, induce changes in size distributions in clouds by
transferring mass from small to large droplets in radiatively cooling clouds which have no net growth,
the magnitude of the effect being much greater for larger droplets than the Ostwald ripening process.
- [Gelbard 1990](https://doi.org/10.1080/02786829008959355)   
  "_Modeling Multicomponent Aerosol Particle GrowthBy Vapor Condensation_"  
- [Kulmala et al. 1992](https://doi.org/10.1016/0021-8502%2892%2990362-Y)   
  "_Activation and growth of cloud condensation nuclei by binary nucleation and condensation processes_"
  > After about ten seconds the condensation rate has increased so much that the water saturation passes its maximum and particles which have not reached a size large enough start to evaporate due to the Kelvin effect (Ostwald ripening).
- [Srivastava 1991](http://doi.org/10.1175/1520-0469%281991%29048<1596%3AGOCDBC>2.0.CO%3B2)   
  "_Growth of Cloud Drops by Condensation: Effect of Surface Tension on the Dispersion of Drop Sizes_"
- [Sagui & Desai 1995 (Phys. Rev. Lett. 74)](https://doi.org/10.1103/PhysRevLett.74.1119)   
  "_Ostwald Ripening in Systems with Competing Interactions_"
- [Vesala 1997](https://doi.org/10.1016/S0021-8502(96)00461-2)    
  "_Models for condensational growth and evaporation of binary aerosol particles_"
- [Çelik and Marwitz 1999](https://doi.org/10.1175/1520-0469%281999%29056%3C3091:DSBBRP%3E2.0.CO;2)   
  "_Droplet Spectra Broadening by Ripening Process. Part I: Roles of Curvature and Salinity of Cloud Droplets_"
- [Wood et al. 2002](https://doi.org/10.1175/1520-0469%282002%29059%3C2681:HIITSR%3E2.0.CO;2)    
  "_How Important Is the Spectral Ripening Effect in Stratiform Boundary Layer Clouds? Studies Using Simple Trajectory Analysis_"
  > Recent modeling efforts (Celik and Marwitz 1999, henceforth CM99) have shown that parcels of air that remain within a cloud for significant periods of time undergo a ripening process whereby large droplets can grow at the expense of smaller droplets. This has been termed “Ostwald ripening,” following pioneering observations of crystals in 1896 by W. Ostwald. Because larger droplets grow and smaller droplets shrink, the effect of this process is to broaden the spectrum at both small and large sizes. Srivastava (1991) demonstrates, using the droplet growth equation, that the difference between the squared radii of any two activated nuclei is expected to increase with time, rather than remain constant, if the effect of droplet surface tension is included.
- [Pego 2007](http://www.math.cmu.edu/CNA/Publications/publications2006/001abs/001abs.html)   
  "_Lectures on dynamics in models of coarsening and coagulation_"    
  > One of the classic dynamic scaling phenomenaobserved in material systems is Ostwald ripening, a process that occursduring the condensation of a supersaturated vapor, for example (think of clouds or fog), or during phase separation in metallic alloys.
- [Lewellen 2012](https://doi.org/10.1175/JAS-D-11-029.1)    
  "_Analytic Solutions for Evolving Size Distributions of Spherical Crystals or Droplets Undergoing Diffusional Growth in Different Regimes_"   
  > In the limit of zero external forcing on the parcel the size-spectrum solution is mathematically equivalent to a classic result in the theory of Ostwald ripening of solid solutions. It is argued that the slow-growth regime may be important in the evolution of contrail cirrus and perhaps in setting upper limits on droplet number densities in stratiform boundary layer clouds. 
- [Baumgartner and Spichtinger 2018](https://doi.org/10.5194/acp-18-2525-2018)   
  "_Towards a bulk approach to local interactions of hydrometeors_"   
  > Note that the Wegener–Bergeron–Findeisen process is different from the Ostwald ripening, in which bigger particles grow at the expense of smaller particles due to the curvature dependency of the saturation pressure
- [Yang_et_al_2018 (Atmos. Chem. Phys. 18)](http://doi.org/10.5194/acp-18-7313-2018)   
  "_Cloud droplet size distribution broadening during diffusional growth: ripening amplified by deactivation and reactivation_"   
  > Results show that the CDSD can be broadened during condensational growth as a result of Ostwald ripening amplified by droplet deactivation and reactivation
- [Chandrakar et al. 2019 (QJRMS)](https://doi.org/10.1002/qj.3692)   
  "_Droplet size distributions in turbulent clouds: experimental evaluation of theoretical distributions_"
  > ... optimal distribution breaks down for broad aerosol size distributions, primarily due to deviations from the measured large-droplet tail. A possible explanation for the deviation is the Ostwald ripening effect coupled with deactivation/activation in polluted cloud conditions
- [Grabowski & Pawlowska 2023](https://doi.org/10.1029/2022GL101917)    
  "_Adiabatic Evolution of Cloud Droplet Spectral Width: A New Look at an Old Problem_"
  > Red lines show CCN classes that become activated and subsequently deactivated. 